In [ ]:
import requests
import pandas as pd
import datetime as datetime
from dateutil import relativedelta
from collections import Counter , defaultdict 
from gazpacho import get , Soup
from collections import OrderedDict
import bs4 as BeautifulSoup
import urllib.request 
import DBcm

In [ ]:
url = 'https://en.wikipedia.org/wiki/Taoiseach'
html = requests.get(url)
total_data = pd.read_html(html.text)
taoiseach_data = total_data[3]
df = taoiseach_data[1:]
df = df.drop(columns=[1])
df = df.drop([1,10,11])
df2 = df.drop(columns=[8])
df2['Name_of_the_President'] = df2[2].str.split('(', expand = True)[0]
df2['Constituency_1'] = df2[2].str.split('for ', expand = True)[1].str.split('until',expand = True)[0].str.strip(' ')
df2['Constituency_2'] = df2[2].str.split('for ', expand = True)[2].str.split('from', expand = True)[0].str.strip(' ')
df2['Term_Start'] = df2[3].str.split('[', expand = True)[0].str.strip(" ")
df2['Term_End'] = df2[4].str.split('[', expand = True)[0].str.strip(" ")
df2 = df2.drop(columns=[2,3,4])
df2.rename(columns ={ 9:'Vice_President' , 10:'Dail' , 5:'Party'} , inplace = True)
df2 = df2.drop(columns=[0,6,7])
df2['Dail'] = df2['Dail'].str.split('(', expand = True)[0]

In [ ]:
df2.loc[df2.Term_End== 'Incumbent', 'Term_End'] =  str(datetime.datetime.today()).split()[0]

In [ ]:
df2.Term_Start = pd.to_datetime(df2.Term_Start)
df2.Term_End = pd.to_datetime(df2.Term_End , errors='coerce')

In [ ]:
URL = "https://en.wikipedia.org/wiki/Taoiseach" 
html = get(URL)
soup = Soup(html)
tables = soup.find("table")
needed_table = tables[2]
rows = needed_table.find("b")

In [ ]:
link_list = []
for i in rows:
    x = i.find("a")
    link_list.append(x)

In [ ]:
my_final_urls = []
for j in rows:
    x = j.find("a")
    pre_link = x.attrs['href']
    final_link = "https://en.wikipedia.org"+ pre_link
    my_final_urls.append(final_link)

In [ ]:
my_final_urls = sorted(set(my_final_urls))

In [ ]:
my_list = OrderedDict()
for i,j in enumerate(my_final_urls):
    my_list[i] = j

In [ ]:
df2["birthday_coded"] = [13,13,13,13,13,0,0,0,0,0,0,0,8,0,8,0,12,12,12,7,7,11,7,4,6,4,6,6,4,4,4,1,1,1,9,2,2,2,2,3,5,5,5,10,10]

In [ ]:
mpt_list = []
for i in df2['birthday_coded']:
    df2['birthday_coded'] = my_list[i]
    final_url = urllib.request.urlopen(my_list[i])
    soup = BeautifulSoup.BeautifulSoup(final_url , 'lxml')
    var = soup.find("table" , attrs = {'class' : 'infobox vcard'})
    birth_day = var.find('span' , attrs = {'class' : 'bday'})
    mpt_list.append(birth_day.text)
df2["BirthDate"] = mpt_list 
df2 = df2.drop(columns=['birthday_coded'])
df2.reset_index(drop=True, inplace=True)
df2.BirthDate = pd.to_datetime(df2.BirthDate)

In [ ]:
database_input = df2.values.tolist()

# Before Running the following code, need to run below queries in MySQL for the creation of user , database and its tables

CREATE USER 'aditya'@'localhost' IDENTIFIED BY 'aditya'; grant all privileges on presidents.* to 'aditya'@'localhost'; flush privileges; quit

--> Now login with user aditya:

mysql -u aditya -p

use database presidents;

create table table_1( -> Party varchar(50), -> Vice_President varchar(70), -> Dail int, -> Name_of_the_President varchar(100), -> Constituency_1 varchar(60), -> Constituency_2 varchar(60), -> Term_Start date, -> Term_End date, -> BirthDate date);

In [ ]:
config = {
    'user': 'aditya',
    'password': 'aditya',
    'host': '127.0.0.1',
    'database': 'presidents',
}

In [ ]:
with DBcm.UseDatabase(config) as cursor:
    SQL = """INSERT INTO table_1 (
    Party,Vice_President,Dail,Name_of_the_President,Constituency_1,Constituency_2,Term_Start,Term_End,BirthDate) VALUES (%s, %s, %s, %s , %s, %s, %s, %s , %s )"""
    cursor.executemany(SQL, database_input )

##  (a) Provide a list of constituencies in Ireland which have produced a Taoiseach.

In [ ]:
answerA = []
A_answer =[]
for i in df2["Constituency_1"]:
    answerA.append(i)
for j in df2["Constituency_2"]: 
    answerA.append(j)
for k in set(answerA):
    if (bool(k)) != False:
        A_answer.append(k)
A_answer                  

## (b) Which political party has produced the most individual Taoiseachs?

In [ ]:
with DBcm.UseDatabase(config) as cursor:
    SQL = "select Party , count(DISTINCT Name_of_the_President) from table_1 group by party order by count(DISTINCT Name_of_the_President) DESC limit 1 "
    cursor.execute(SQL)
    output = cursor.fetchall()
    print(output)

## (c) Provide the list of names for those politicians who previously held the position of
## “Vice President” or “Tánaiste” prior to becoming Taoiseach.

In [ ]:
with DBcm.UseDatabase(config) as cursor:
    SQL = "select distinct Name_of_the_President from table_1 where  Name_of_the_President in (select Vice_President from table_1)"
    cursor.execute(SQL)
    output = cursor.fetchall()
    print(output)

## (d) Which political party held the office of Taoiseach for the longest amount of overall time, and for how long (in years, months, and days)?

In [ ]:
var_1 = defaultdict(list)
for index, row in df2.iterrows():
    
    a = row['Term_End']
    b = row['Term_Start']
    c = row['Party']
    
    diff = relativedelta.relativedelta(a,b)
   
    years = int(diff.years)
    months = int(diff.months) 
    days = int(diff.days)
      
    x = years*365 + months*12 + days
    
    var_1[c].append(x)
new_dict1= {}
for g,h in var_1.items():
    new_dict1[g] = set(h) 
a = new_dict1['Fianna Fáil'] 
b = new_dict1['Fine Gael']
sum_Fianna_Fáil = sum(a)
sum_Fine_Gael = sum(b)
print(sum_Fianna_Fáil)    

## (g) Which political party has held the office of Taoiseach for the largest number of Dáils?

In [ ]:
with DBcm.UseDatabase(config) as cursor:
    SQL = "select count(DISTINCT Dail) , Party  from table_1 group by Party  order by count(DISTINCT Dail) DESC limit 1"
    cursor.execute(SQL)
    output = cursor.fetchall()
    print(output)

## (h) Which individual politician has held the office of Taoiseach for the most number of Dáils?

In [ ]:
with DBcm.UseDatabase(config) as cursor:
    SQL = "select count(DISTINCT Dail) , Name_of_the_President  from table_1 group by Name_of_the_President order by count(DISTINCT Dail) DESC limit 1"
    cursor.execute(SQL)
    output = cursor.fetchall()
    print(output)

## (i) What age was each Taoiseach on the day they assumed office? 

In [ ]:
age_f_day = []

for index, row in df2.iterrows():
    
    a = row['Term_Start']
    b = row['BirthDate']
    
    diff = relativedelta.relativedelta(a,b)
   
    years = diff.years
    months = diff.months 
    days = diff.days
   
    age_f_day.append(f"{years}Years{months} Months{days} Days")
answer_i = df2['Name_of_the_President'].copy()
age_f_day_df = pd.DataFrame(age_f_day)
answer_df = pd.concat([answer_i,age_f_day_df],axis=1)
answer_df.rename(columns ={ 0:'AGE'} , inplace = True)
answer_df = answer_df.drop_duplicates()
answer_df

## (j) On the last day of their term of office, which politician was the oldest?

In [ ]:
age_f_day = []

for index, row in df2.iterrows():
    
    a = row['Term_End']
    b = row['BirthDate']
    
    diff = relativedelta.relativedelta(a,b)
   
    years = diff.years
    months = diff.months 
    days = diff.days
   
    age_f_day.append(f"{years}")
answer_i = df2['Name_of_the_President'].copy()
age_f_day_df = pd.DataFrame(age_f_day)
answer_df = pd.concat([answer_i,age_f_day_df],axis=1)
answer_df.rename(columns ={ 0:'AGE'} , inplace = True)
answer_df = answer_df.drop_duplicates(keep='last')
answer_df[answer_df['AGE']==answer_df['AGE'].max()]

## (k) In your view, and based on statistics calculated from your scraped data, which Irishpolitical party is the most successful? Show and describe your calculations.

From the data scraped and queried in the assignment, it is visible that the most successful political parties prevalent in Ireland are Fianna_Fáil and Fine_Gael.
Both the parties have been in power for almost alternatively.